### **1. Importando as bibliotecas e bases de dados**

In [1]:
# Importando as bibliotecas
import sys
import pandas as pd                        
import numpy as np                             
import seaborn as sns                                 
import matplotlib.pyplot as plt   
from wordcloud import WordCloud
               
from sklearn.cluster import KMeans    
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA  
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

- Importando a base original

In [ ]:
base_original = pd.read_csv("../data/data.csv")

#### **Explicações das etapas seguintes:**

- Separando a base em treino e teste, para simular a entrada de dados
- Criando um modelo KMeans de 5 clusters
- Criando um PCA de 1 componente para as colunas: acousticness, energy e loudness, que estão alto correlacionadas. Isso em uma base que tenha muitas features pode ser importante na redução de dimensionalidade.
- Instanciando normalizadores para padronizar a escala das colunas em 0 e 1

In [ ]:
xtreino, xteste = train_test_split(base_original, test_size=0.10, random_state=10)

kmeans1 = KMeans(n_clusters=5, 
                init='k-means++', 
                n_init=50,
                max_iter=300,
                tol = 1e-4,
                random_state=0
                )

pca_acou_ener_loud = PCA(n_components=1)

minmax_year = MinMaxScaler()
minmax_tempo = MinMaxScaler()
minmax_pca = MinMaxScaler()
minmax_loudness = MinMaxScaler()

def selecao_colunas(base):
    base = base[['year','tempo','acousticness','danceability','energy','instrumentalness','liveness','loudness','speechiness','valence']]
    return base

In [ ]:
def selecao_colunas(base):
    base = base[['year','tempo','acousticness','danceability','energy','instrumentalness','liveness','loudness','speechiness','valence']]
    return base

def transformar(base):
    return pd.DataFrame(base)

def selecao_colunas(base):
    base = base[['duration_ms','tempo','acousticness','danceability','energy','instrumentalness','liveness','loudness','speechiness','valence']]
    return base

def retornar_cluster(resultadokmeans):
    if isinstance(resultadokmeans, pd.DataFrame):
        return pd.DataFrame(resultadokmeans.idxmin(axis=1), columns=['Cluster'])
    elif isinstance(resultadokmeans, np.ndarray):
        return pd.DataFrame(np.argmin(resultadokmeans, axis=1), columns=['Cluster'])

In [ ]:
pipeline_preprocessamento = Pipeline([
    ("selecao_colunas", FunctionTransformer(selecao_colunas)),
     
    ("escala_colunas", ColumnTransformer(
        transformers=[
            ("minmax_duration_ms", MinMaxScaler(), ["duration_ms"]),
            ("minmax_tempo", MinMaxScaler(), ["tempo"]),
            ("minmax_loudness", MinMaxScaler(), ["loudness"]),  
        ], remainder='passthrough')),
    
    ("pca", ColumnTransformer(
        transformers=[
            ("pca", PCA(n_components=1), [3, 5]),
        ], remainder='passthrough')),
    
    ("escala_pca", ColumnTransformer(
        transformers=[
            ("minmax_pca", MinMaxScaler(), [0]),  
        ], remainder='passthrough')),
    ])


pipeline_modelo = Pipeline([
        ("clusterizacao", KMeans(n_clusters=5, 
                init='k-means++', 
                n_init=50,
                max_iter=500,
                tol = 1e-4,
                random_state=0
                )),
])

pipeline_completo = Pipeline([
        ("preprocessamento", pipeline_preprocessamento),
        ("modelo",pipeline_modelo),
    ])

a = pd.DataFrame(pipeline_completo.fit_transform(xtreino))